In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from imblearn.under_sampling import RandomUnderSampler
import math

In [ ]:
df = pd.read_csv('attention-analysis/preprocessed_data_extra.csv')

features = [
    'face_movement', 'body_movement', 'eye_openness_rate',
    'eye_direction_x', 'eye_direction_y', 'mouth_openness_rate',
    'yaw_angle', 'pitch_angle', 'roll_angle'
]
INPUT_SIZE = len(features)
SEQ_LEN = 30

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

X_seq, y_seq = [], []
for user_id in df['id'].unique():
    user_df = df[df['id'] == user_id].copy()
    for i in range(len(user_df) - SEQ_LEN):
        seq = user_df[features].iloc[i:i + SEQ_LEN].values
        # Etiket, sekansın son elemanına karşılık gelen değer olarak ayarlandı
        label = user_df['isAttentive'].iloc[i + SEQ_LEN - 1] 
        X_seq.append(seq)
        y_seq.append(int(label))

X_seq, y_seq = np.array(X_seq), np.array(y_seq)
X_flat = X_seq.reshape((X_seq.shape[0], -1))
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_flat, y_seq)
X_seq_balanced = X_resampled.reshape((-1, SEQ_LEN, INPUT_SIZE))
y_seq_balanced = y_resampled

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X_seq_balanced, y_seq_balanced, test_size=0.2, stratify=y_seq_balanced, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, stratify=y_train_val, random_state=42
)

class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class ConvTransformerClassifier(nn.Module):
    def __init__(self, input_size, d_model, n_head, n_layers, dropout, kernel_size, conv_out_channels, num_classes=2):
        super().__init__()
        
        self.conv1d = nn.Conv1d(
            in_channels=input_size, 
            out_channels=conv_out_channels,
            kernel_size=kernel_size,
            padding='same' # Sekans boyutu değişmesin
        )
        self.activation = nn.GELU()
        
        self.projection = nn.Linear(conv_out_channels, d_model)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=n_head, dropout=dropout, batch_first=True,
            dim_feedforward=d_model*4 # Feedforward boyutunu d_model'in 4 katı olarak ayarlamak yaygın bir pratiktir
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        
        self.classifier = nn.Linear(d_model, num_classes)

    def forward(self, x):
        
        x = x.permute(0, 2, 1)
        x = self.conv1d(x)
        x = self.activation(x)
        
        x = x.permute(0, 2, 1)
        
        x = self.projection(x)
        
        x = self.transformer_encoder(x)
        
        x = x.mean(dim=1)
        x = self.classifier(x)
        return x

best_params = {
    'learning_rate': 0.0003281432475028239,
    'd_model': 28,
    'n_heads': 2,
    'n_layers': 2,
    'dropout': 0.1014615298421584,
    'batch_size': 32,
    # Not: Bu parametreler orijinal config'de yoktu, makul varsayılanlar seçildi.
    'kernel_size': 5,
    'conv_out_channels': 24
}

if best_params['d_model'] % best_params['n_heads'] != 0:
    raise ValueError("d_model, n_heads'e tam bölünmelidir!")

print(best_params)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_train_final = np.concatenate((X_train, X_val), axis=0)
y_train_final = np.concatenate((y_train, y_val), axis=0)
final_train_dataset = SequenceDataset(X_train_final, y_train_final)
val_dataset = SequenceDataset(X_val, y_val)
test_dataset = SequenceDataset(X_test, y_test)

final_train_loader = DataLoader(final_train_dataset, batch_size=best_params['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'])
test_loader = DataLoader(test_dataset, batch_size=best_params['batch_size'])

final_model = ConvTransformerClassifier(
    input_size=INPUT_SIZE,
    d_model=best_params['d_model'],
    n_head=best_params['n_heads'],
    n_layers=best_params['n_layers'],
    dropout=best_params['dropout'],
    kernel_size=best_params['kernel_size'],
    conv_out_channels=best_params['conv_out_channels']
).to(device)

optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params['learning_rate'])
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=10, verbose=True)

train_losses = []
val_losses = []
FINAL_EPOCHS = 120

for epoch in range(FINAL_EPOCHS):
    # Eğitim Aşaması
    final_model.train()
    total_train_loss = 0
    for xb, yb in final_train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = final_model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()
    avg_train_loss = total_train_loss / len(final_train_loader)
    train_losses.append(avg_train_loss)
    
    final_model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = final_model(xb)
            loss = criterion(pred, yb)
            total_val_loss += loss.item()
    avg_val_loss = total_val_loss / len(val_loader)
    val_losses.append(avg_val_loss)

    scheduler.step(avg_val_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:03d}/{FINAL_EPOCHS} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(train_losses, label='Eğitim Kaybı (Training Loss)')
plt.plot(val_losses, label='Doğrulama Kaybı (Validation Loss)')
plt.title('Eğitim ve Doğrulama Kaybı Grafiği (Convolutional Transformer)')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

final_model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        outputs = final_model(xb)
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        y_true.extend(yb.numpy())
        y_pred.extend(preds)

print(classification_report(y_true, y_pred, target_names=['Dikkatsiz (0)', 'Dikkatli (1)']))
print("\nConfusion Matrix:")
print(confusion_matrix(y_true, y_pred))

# --- 7. Model ve Scaler'ı Kaydetme ---
print("\n--- Model ve Scaler kaydediliyor... ---")
torch.save(final_model.state_dict(), "conv_transformer_model.pt")
joblib.dump(scaler, "minmax_scaler_conv.pkl")
print("Model 'conv_transformer_model.pt' olarak kaydedildi.")
print("Scaler 'minmax_scaler_conv.pkl' olarak kaydedildi.")

               precision    recall  f1-score   support

Dikkatsiz (0)       0.86      0.83      0.85       144
 Dikkatli (1)       0.84      0.86      0.85       143

     accuracy                           0.85       287
    macro avg       0.85      0.85      0.85       287
 weighted avg       0.85      0.85      0.85       287


Confusion Matrix:
[[120  24]
 [ 20 123]]